SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (2103138248.py, line 1)

In [2]:
import pandas as pd
import re

In [3]:
# Change this path if needed
df = pd.read_csv("C:/Users/sayan/Downloads/imdb_query_execution_times_complex_10k.csv")

# Show first few rows
print(df.head())

                                               query  execution_time
0  SELECT tb.tconst, tb.primaryTitle, tr.averageR...        0.003061
1  SELECT nb.primaryName, tb.primaryTitle FROM na...        0.000158
2  SELECT nb.primaryName, AVG(tr.averageRating) A...        0.000105
3  SELECT nb.primaryName AS directorName, COUNT(*...        0.000158
4  SELECT te.tconst, te.parentTconst, tb.primaryT...        0.000150


In [4]:
sql_keywords = [
    # Core DML / DDL
    "SELECT", "INSERT", "UPDATE", "DELETE", "CREATE", "ALTER", "DROP", "TRUNCATE",
    "FROM", "WHERE", "GROUP BY", "ORDER BY", "HAVING", "LIMIT", "OFFSET",
    "DISTINCT", "AS", "INTO", "VALUES", "SET",

    # Joins
    "JOIN", "INNER JOIN", "LEFT JOIN", "RIGHT JOIN", "FULL JOIN", "CROSS JOIN",
    "UNION", "UNION ALL", "INTERSECT", "EXCEPT",

    # Aggregates
    "COUNT", "SUM", "AVG", "MIN", "MAX",

    # Conditions
    "AND", "OR", "NOT", "IN", "BETWEEN", "LIKE", "IS NULL", "IS NOT NULL",
    "EXISTS", "CASE", "WHEN", "THEN", "ELSE", "END",

    # Window / set
    "ALL", "ANY", "SOME", "OVER", "PARTITION BY", "ROW_NUMBER", "RANK", "DENSE_RANK",

    # Schema / transaction
    "DATABASE", "TABLE", "INDEX", "VIEW", "TRIGGER", "SEQUENCE",
    "PRIMARY KEY", "FOREIGN KEY", "CHECK", "DEFAULT", "CONSTRAINT",
    "BEGIN", "COMMIT", "ROLLBACK",

    # Misc
    "EXPLAIN", "ANALYZE", "DESCRIBE", "SHOW", "USE"
]


In [5]:
def extract_sql_features(query):
    if pd.isna(query):
        return {}
    q = str(query).upper()
    features = {}

    # One-hot encode all keywords
    for kw in sql_keywords:
        kw_regex = r"\b" + re.escape(kw) + r"\b"
        features[kw.replace(" ", "_")] = int(bool(re.search(kw_regex, q)))

    # Number of attributes
    match = re.search(r"SELECT (.*?) FROM", q, re.DOTALL)
    if match:
        attrs = [a.strip() for a in match.group(1).split(',')]
        features["num_attributes"] = len([a for a in attrs if a and a != '*'])
    else:
        features["num_attributes"] = 0

    # Number of tables
    match = re.search(r"FROM (.*?)( WHERE| GROUP BY| ORDER BY|;|$)", q, re.DOTALL)
    if match:
        tables = [t.strip() for t in match.group(1).split(',')]
        features["num_tables"] = len([t for t in tables if t])
    else:
        features["num_tables"] = 0

    # Number of conditions
    features["num_conditions"] = len(re.findall(r"\bWHERE\b|\bAND\b|\bOR\b", q))

    # Number of joins
    features["num_joins"] = len(re.findall(r"\bJOIN\b", q))

    # Subquery flag
    features["has_subquery"] = int(len(re.findall(r"\(SELECT ", q)) > 0)

    # Total keyword count
    features["num_keywords"] = sum(
        features[k] for k in features if k in [kw.replace(" ", "_") for kw in sql_keywords]
    )

    return features


In [10]:
# Replace 'query' with your actual column name if different
df["features"] = df["query"].apply(extract_sql_features)

# Expand the dict column into separate columns
features_df = pd.DataFrame(df["features"].tolist())

# Combine with the original dataset
final_df = pd.concat([df, features_df], axis=1)

# Save to new CSV
final_df.to_csv("C:/Users/sayan/Desktop/New folder (3)/sql_features_encoded.csv", index=False)

print("✅ Feature encoding complete. File saved as 'sql_features_encoded.csv'")


✅ Feature encoding complete. File saved as 'sql_features_encoded.csv'


In [9]:
final_df

,query,execution_time,features,SELECT,INSERT,UPDATE,DELETE,CREATE,ALTER,DROP,...,ANALYZE,DESCRIBE,SHOW,USE,num_attributes,num_tables,num_conditions,num_joins,has_subquery,num_keywords
0,"SELECT tb.tconst, tb.primaryTitle, tr.averageR...",0.003061,"{'SELECT': 1, 'INSERT': 0, 'UPDATE': 0, 'DELET...",1,0,0,0,0,0,0,...,0,0,0,0,3,1,1,1,0,6
1,"SELECT nb.primaryName, tb.primaryTitle FROM na...",0.000158,"{'SELECT': 1, 'INSERT': 0, 'UPDATE': 0, 'DELET...",1,0,0,0,0,0,0,...,0,0,0,0,2,1,2,2,0,6
2,"SELECT nb.primaryName, AVG(tr.averageRating) A...",0.000105,"{'SELECT': 1, 'INSERT': 0, 'UPDATE': 0, 'DELET...",1,0,0,0,0,0,0,...,0,0,0,0,2,1,0,2,0,7
3,"SELECT nb.primaryName AS directorName, COUNT(*...",0.000158,"{'SELECT': 1, 'INSERT': 0, 'UPDATE': 0, 'DELET...",1,0,0,0,0,0,0,...,0,0,0,0,2,1,0,2,0,7
4,"SELECT te.tconst, te.parentTconst, tb.primaryT...",0.000150,"{'SELECT': 1, 'INSERT': 0, 'UPDATE': 0, 'DELET...",1,0,0,0,0,0,0,...,0,0,0,0,4,1,1,2,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,"SELECT tconst, primaryTitle, runtimeMinutes FR...",0.000029,"{'SELECT': 1, 'INSERT': 0, 'UPDATE': 0, 'DELET...",1,0,0,0,0,0,0,...,0,0,0,0,3,1,2,0,1,6
9996,SELECT nb.primaryName FROM name_basics nb WHER...,0.000013,"{'SELECT': 1, 'INSERT': 0, 'UPDATE': 0, 'DELET...",1,0,0,0,0,0,0,...,0,0,0,0,1,1,3,0,1,7
9997,"SELECT tp.category, COUNT(*) as num_people, AV...",0.000050,"{'SELECT': 1, 'INSERT': 0, 'UPDATE': 0, 'DELET...",1,0,0,0,0,0,0,...,0,0,0,0,3,1,0,1,0,8
9998,"SELECT tb.startYear, SUBSTR(tb.genres, 1, INST...",0.000029,"{'SELECT': 1, 'INSERT': 0, 'UPDATE': 0, 'DELET...",1,0,0,0,0,0,0,...,0,0,0,0,7,1,1,0,0,9


In [11]:
# Extract all keyword columns (ignore numeric feature columns)
keyword_columns = [col for col in final_df.columns if col in [kw.replace(" ", "_") for kw in sql_keywords]]

# Sum each keyword column
keyword_usage = final_df[keyword_columns].sum()

# Find which keywords were never used
unused_keywords = keyword_usage[keyword_usage == 0].index.tolist()

print("🔍 Keywords never used in any query:")
print(unused_keywords)

print(f"\nTotal unused keywords: {len(unused_keywords)} out of {len(keyword_columns)}")


🔍 Keywords never used in any query:
['INSERT', 'UPDATE', 'DELETE', 'CREATE', 'ALTER', 'DROP', 'TRUNCATE', 'OFFSET', 'DISTINCT', 'INTO', 'VALUES', 'SET', 'INNER_JOIN', 'RIGHT_JOIN', 'FULL_JOIN', 'CROSS_JOIN', 'UNION_ALL', 'INTERSECT', 'EXCEPT', 'SUM', 'MIN', 'MAX', 'OR', 'IN', 'BETWEEN', 'LIKE', 'CASE', 'WHEN', 'THEN', 'ELSE', 'END', 'ALL', 'ANY', 'SOME', 'OVER', 'PARTITION_BY', 'ROW_NUMBER', 'RANK', 'DENSE_RANK', 'DATABASE', 'TABLE', 'INDEX', 'VIEW', 'TRIGGER', 'SEQUENCE', 'PRIMARY_KEY', 'FOREIGN_KEY', 'CHECK', 'DEFAULT', 'CONSTRAINT', 'BEGIN', 'COMMIT', 'ROLLBACK', 'EXPLAIN', 'ANALYZE', 'DESCRIBE', 'SHOW', 'USE']

Total unused keywords: 58 out of 76
